# Notebook 14: Scaled Thermal Convection

This notebook demonstrates a complete thermal convection problem with non-dimensional scaling. We'll solve a coupled Stokes-temperature system with realistic mantle convection parameters.

In this notebook you'll learn:
- Setting up coupled Stokes and temperature equations
- Computing buoyancy forces from temperature
- Using realistic mantle convection parameters
- Interpreting non-dimensional results in dimensional units
- Understanding the Rayleigh number

In [1]:
import nest_asyncio

nest_asyncio.apply()

import os

os.environ["SYMPY_USE_CACHE"] = "no"

import numpy as np
import sympy
import underworld3 as uw

## Problem Setup: Steady-State Thermal Convection

We'll solve thermal convection in a square box with:
- Hot bottom boundary (driving convection)
- Cold top boundary
- Free-slip walls
- Incompressible Stokes flow

The equations are:
- **Stokes**: ∇·σ = -ρ₀αgT (buoyancy from temperature)
- **Energy**: u·∇T = κ∇²T (steady-state, advection = diffusion)

## Reference Quantities and Dimensional Parameters

First, set up realistic mantle convection parameters and reference scales:

In [2]:
# Reset and get model
uw.reset_default_model()
model = uw.get_default_model()

# Realistic mantle parameters
model.set_reference_quantities(
    domain_depth=uw.quantity(2900, "km"),  # Mantle depth
    plate_velocity=uw.quantity(5, "cm/year"),  # Plate motion
    mantle_viscosity=uw.quantity(1e21, "Pa*s"),  # Mantle viscosity
    temperature_difference=uw.quantity(3000, "K"),  # ΔT across mantle
)

# Physical constants (dimensional)
rho0 = 3300 * uw.units("kg/m^3")  # Reference density
alpha = 3e-5 * uw.units("1/K")  # Thermal expansion
g = 10 * uw.units("m/s^2")  # Gravity
kappa = 1e-6 * uw.units("m^2/s")  # Thermal diffusivity

# Display parameters
rho0, alpha, g, kappa

(<Quantity(3300.0, 'kilogram / meter ** 3')>,
 <Quantity(3e-05, '1 / kelvin')>,
 <Quantity(10.0, 'meter / second ** 2')>,
 <Quantity(1e-06, 'meter ** 2 / second')>)

## Compute Rayleigh Number

The Rayleigh number Ra characterizes the vigor of convection:

Ra = (ρ₀ α g ΔT L³) / (η κ)

Higher Ra means more vigorous convection.

In [3]:
rho0 = uw.expression(r"\rho_0", uw.quantity(3300, "kg/m^3"), "reference density")
alpha = uw.expression(r"\alpha", uw.quantity(3e-5, "1/K"), "thermal expansivity")
g = uw.expression(r"g", uw.quantity(9.8, "m/s/s"), "gravitational acceleration")
kappa = uw.expression(r"\kappa", uw.quantity(1e-6, "m^2/s"), "thermal diffusivity")
L0 = uw.expression(r"L_0", uw.quantity(2900, "km").to("m"), "reference length, mantle depth")
eta0 = uw.expression(r"\eta_0", uw.quantity(1e21, "Pa*s"), "Reference viscosity") 
DeltaT = uw.expression(r"\Delta T", uw.quantity(3000, "K"), "Temperature drop")

# Compute Rayleigh number: Ra = (ρ₀ α g ΔT L³) / (η κ)

Ra = uw.expression(r"\mathrm{Ra}", (rho0 * alpha * g * DeltaT * L0**3) / (eta0 * kappa), "Rayleigh number")


In [4]:
print(uw.unwrap(Ra, keep_constants=True))

80.4837*\Delta T*\alpha*g/\kappa


In [5]:
print(kappa * alpha)

1.0e-6*\alpha


This is a moderate Rayleigh number typical of Earth's mantle, producing steady convection cells.

## Create Mesh and Variables

Set up the computational domain and fields:

In [6]:
# Create mesh (non-dimensional coordinates 0 to 1)
resolution = 16
mesh = uw.meshing.StructuredQuadBox(
    elementRes=(resolution, resolution),
    minCoords=(0.0, 0.0),
    maxCoords=(1.0, 1.0),
    qdegree=3,
)

# Get coordinate symbols
x, y = mesh.X


Structured box element resolution 16 16


In [7]:
# Create variables with physical units
v = uw.discretisation.MeshVariable("v", mesh, 2, degree=2, units="m/s")
p = uw.discretisation.MeshVariable("p", mesh, 1, degree=1, units="Pa")
T = uw.discretisation.MeshVariable("T", mesh, 1, degree=2, units="K")

# Get scaling coefficients
V0_scale = v.scaling_coefficient
P0_scale = p.scaling_coefficient
T0_scale = T.scaling_coefficient

print(f"Velocity scale: {V0_scale:.3e} m/s")
print(f"Pressure scale: {P0_scale:.3e} Pa")
print(f"Temperature scale: {T0_scale:.3e} K")

Velocity scale: 1.584e-09 m/s
Pressure scale: 5.463e+05 Pa
Temperature scale: 3.000e+03 K


## Enable Non-Dimensional Scaling

In [8]:
uw.use_nondimensional_scaling(True)

In [9]:
# Create Stokes solver
stokes = uw.systems.Stokes(mesh, velocityField=v, pressureField=p)
stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel

# IMPORTANT: Since we have reference quantities defined, we must provide units
# for all constitutive model parameters. The viscosity is in the stress tensor: σ = 2ηε̇
# In dimensional form, η has units of Pa·s (pascal-seconds)
stokes.constitutive_model.Parameters.viscosity = uw.quantity(1e21, "Pa*s")

# Buoyancy force (ND form)
# In non-dimensional form: F̂ = -Ra·T̂ where Ra contains all physical constants
unit_y = sympy.Matrix([0, 1])
stokes.bodyforce = -Ra * T[0] * unit_y

# Boundary conditions: free slip on all walls
stokes.add_dirichlet_bc((0.0, sympy.oo), "Left")  # u_x = 0 on left (u_y free)
stokes.add_dirichlet_bc((0.0, sympy.oo), "Right")  # u_x = 0 on right (u_y free)
stokes.add_dirichlet_bc((sympy.oo, 0.0), "Top")  # u_y = 0 on top (u_x free)
stokes.add_dirichlet_bc((sympy.oo, 0.0), "Bottom")  # u_y = 0 on bottom (u_x free)

stokes

**Class**: <class 'underworld3.systems.solvers.SNES_Stokes'>

# Underworld / PETSc General Saddle Point Equation Solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Boundary Conditions

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Left | $\left[\begin{matrix}0.0 & \infty\end{matrix}\right]  $ | 
| **essential** | Right | $\left[\begin{matrix}0.0 & \infty\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}\infty & 0.0\end{matrix}\right]  $ | 
| **essential** | Bottom | $\left[\begin{matrix}\infty & 0.0\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

In [10]:
# Temperature: T = (1 - y) + 0.1*cos(2πx)*sin(πy)
# This is in ND units (0 to 1), representing (T-T_cold)/ΔT
T_init = (1.0 - y) + 0.1 * sympy.cos(2 * np.pi * x) * sympy.sin(np.pi * y)

# Evaluate and assign
T.array[...] = uw.function.evaluate(T_init, T.coords).reshape(T.array.shape)

print(f"Initial temperature (ND): [{T.min():.3e}, {T.max():.3e}]")

Initial temperature (ND): [-7.401e-20 kelvin, 3.333e-04 kelvin]


In [11]:
# Create advection-diffusion solver
adv_diff = uw.systems.AdvDiffusionSLCN(
    mesh,
    u_Field=T,
    V_fn=v,
)

# Constitutive model for diffusion
# IMPORTANT: Since we have reference quantities defined, we must provide units
# for all parameters. The diffusivity is in the flux: flux = -κ∇T
# In dimensional form, κ has units of m²/s (thermal diffusivity)
adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = uw.quantity(1e-6, "m^2/s")

# Temperature boundary conditions
# These are non-dimensional (ND values between 0 and 1)
adv_diff.add_dirichlet_bc(1.0, "Bottom")  # Hot bottom (T̂ = 1)
adv_diff.add_dirichlet_bc(0.0, "Top")  # Cold top (T̂ = 0)

adv_diff

**Class**: <class 'underworld3.systems.solvers.SNES_AdvectionDiffusion'>

# Underworld / PETSc General Scalar Equation Solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Bottom | $\left[\begin{matrix}0.000333333333333333\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}0.0\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [12]:
# Create Stokes solver
stokes = uw.systems.Stokes(mesh, velocityField=v, pressureField=p)
stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel

# IMPORTANT: Since we have reference quantities defined, we must provide units
# for all constitutive model parameters. The viscosity is in the stress tensor: σ = 2ηε̇
# In dimensional form, η has units of Pa·s (pascal-seconds)
stokes.constitutive_model.Parameters.viscosity = uw.quantity(1e21, "Pa*s")

# Buoyancy force (ND form)
# In non-dimensional form: F̂ = -Ra·T̂ where Ra contains all physical constants
unit_y = sympy.Matrix([0, 1])
stokes.bodyforce = -Ra * T[0] * unit_y

# Boundary conditions: free slip on all walls
stokes.add_dirichlet_bc((0.0, sympy.oo), "Left")  # u_x = 0 on left (u_y free)
stokes.add_dirichlet_bc((0.0, sympy.oo), "Right")  # u_x = 0 on right (u_y free)
stokes.add_dirichlet_bc((sympy.oo, 0.0), "Top")  # u_y = 0 on top (u_x free)
stokes.add_dirichlet_bc((sympy.oo, 0.0), "Bottom")  # u_y = 0 on bottom (u_x free)

stokes


**Class**: <class 'underworld3.systems.solvers.SNES_Stokes'>

# Underworld / PETSc General Saddle Point Equation Solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Boundary Conditions

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Left | $\left[\begin{matrix}0.0 & \infty\end{matrix}\right]  $ | 
| **essential** | Right | $\left[\begin{matrix}0.0 & \infty\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}\infty & 0.0\end{matrix}\right]  $ | 
| **essential** | Bottom | $\left[\begin{matrix}\infty & 0.0\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

In [13]:
display(sympy.Matrix((Ra*v[0], Ra*v[1])))

Matrix([
[80.4837*\Delta T*\alpha*g*{ \hspace{ 0.002pt } {v} }_{ 0 }(N.x, N.y)/\kappa],
[80.4837*\Delta T*\alpha*g*{ \hspace{ 0.002pt } {v} }_{ 1 }(N.x, N.y)/\kappa]])

In [14]:
print((2 * Ra) * v / 2)

Matrix([[80.4837*\Delta T*\alpha*g*{ \hspace{ 0.002pt } {v} }_{ 0 }(N.x, N.y)/\kappa, 80.4837*\Delta T*\alpha*g*{ \hspace{ 0.002pt } {v} }_{ 1 }(N.x, N.y)/\kappa]])


In [15]:
Ra.sym * v

Matrix([[80.4837*\Delta T*\alpha*g*{ \hspace{ 0.002pt } {v} }_{ 0 }(N.x, N.y)/\kappa, 80.4837*\Delta T*\alpha*g*{ \hspace{ 0.002pt } {v} }_{ 1 }(N.x, N.y)/\kappa]])

## Set Up Temperature Solver

For steady-state, we solve the advection-diffusion balance:

v·∇T = κ∇²T

In ND form with Péclet number Pe = VL/κ:

v̂·∇̂T̂ = ∇̂²T̂

In [16]:
# Create advection-diffusion solver
adv_diff = uw.systems.AdvDiffusionSLCN(
    mesh,
    u_Field=T,
    V_fn=v,
)

# Constitutive model for diffusion
# IMPORTANT: Since we have reference quantities defined, we must provide units
# for all parameters. The diffusivity is in the flux: flux = -κ∇T
# In dimensional form, κ has units of m²/s (thermal diffusivity)
adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = uw.quantity(1e-6, "m^2/s")

# Temperature boundary conditions
# These are non-dimensional (ND values between 0 and 1)
adv_diff.add_dirichlet_bc(1.0, "Bottom")  # Hot bottom (T̂ = 1)
adv_diff.add_dirichlet_bc(0.0, "Top")  # Cold top (T̂ = 0)

adv_diff


**Class**: <class 'underworld3.systems.solvers.SNES_AdvectionDiffusion'>

# Underworld / PETSc General Scalar Equation Solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Bottom | $\left[\begin{matrix}0.000333333333333333\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}0.0\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Solve Coupled System Iteratively

Since temperature affects velocity (buoyancy) and velocity affects temperature (advection), we iterate:

1. Solve Stokes for velocity (given temperature)
2. Solve temperature equation (given velocity)
3. Repeat until convergence

In [17]:
stokes.solve()

In [18]:
dt = stokes.estimate_dt() 

In [19]:
model.get_fundamental_scales()

{'length': <Quantity(2900000.0, 'meter')>,
 'temperature': <Quantity(3000.0, 'kelvin')>,
 'time': <Quantity(1.8303408e+15, 'second')>,
 'mass': <Quantity(5.30798832e+42, 'kilogram')>}

In [20]:
adv_diff.solve(timestep=uw.quantity(1e7, "year"))

[0]PETSC ERROR: --------------------- Error Message --------------------------------------------------------------
[0]PETSC ERROR: PETSc has generated inconsistent data
[0]PETSC ERROR: Point 0: -1.42085e+10 1.40692e+10 0. not located in mesh
[0]PETSC ERROR: WARNING! There are unused option(s) set! Could be the program crashed before usage or a spelling mistake, etc!
[0]PETSC ERROR:   Option left: name:-dm_plex_hash_location (no value) source: code
[0]PETSC ERROR:   Option left: name:-f value: /Users/lmoresi/Library/Jupyter/runtime/kernel-e8483652-1667-4ccf-81f4-37aa866346d8.json source: command line
[0]PETSC ERROR:   Option left: name:-options_left value: 0 source: code
[0]PETSC ERROR:   Option left: name:-Solver_108_mg_levels_ksp_converged_maxits (no value) source: code
[0]PETSC ERROR:   Option left: name:-Solver_108_mg_levels_ksp_max_it value: 3 source: code
[0]PETSC ERROR:   Option left: name:-Solver_108_pc_mg_type value: additive source: code
[0]PETSC ERROR:   Option left: name:-So

RuntimeError: Error encountered when trying to interpolate mesh variable.
Interpolation location is possibly outside the domain.

In [ ]:
for iteration in range(10):
    # Solve Stokes (velocity from temperature)
    stokes.solve()

    # Solve temperature (steady-state with current velocity)
    # For steady-state, use a large time step to reach equilibrium
    adv_diff.solve(timestep=uw.quantity(1e9, "year"))

    # Check velocity magnitude
    # Note: v.max() returns tuple (vx_max, vy_max) for vector variables
    v_max_nd = max(v.max().to_non)  # Get largest component
    if iteration % 2 == 0:
        print(f"Iteration {iteration}: max velocity (ND) = {v_max_nd:.3e}")

print("\nConverged solution")

In [ ]:
uw.non_dimensionalise(v.max())

## Examine Non-Dimensional Results

The solution is stored in ND form. Let's look at the ND values:

In [ ]:
# Non-dimensional values (order one, well-conditioned)
# Note: For vector variables, .max() returns tuple (vx_max, vy_max)
v_min_components = v.min()  # Tuple: (vx_min, vy_min)
v_max_components = v.max()  # Tuple: (vx_max, vy_max)

print("Non-dimensional values:")
print(f"  Velocity x-component: [{v_min_components[0]:.3e}, {v_max_components[0]:.3e}]")
print(f"  Velocity y-component: [{v_min_components[1]:.3e}, {v_max_components[1]:.3e}]")
print(f"  Temperature: [{T.min():.3e}, {T.max():.3e}]")

Notice all values are order-one. This is the benefit of ND scaling - PETSc worked with well-conditioned matrices.

## Interpret in Dimensional Units

To get physical values, we need to:
1. Multiply ND values by scaling coefficients
2. Wrap in `uw.quantity()` to attach units
3. Then convert to desired units

In [ ]:
# Get maximum velocity component (ND value is plain float)
v_max_nd = max(v.max())

# Create dimensional Pint quantity
# Note: scaling_coefficient is a plain float, need to wrap with units
v_dimensional = uw.quantity(v_max_nd * V0_scale, "m/s")

# Now can convert units
v_cm_per_year = v_dimensional.to("cm/year")
v_mm_per_year = v_dimensional.to("mm/year")

print(f"Maximum velocity:")
print(f"  {v_cm_per_year:.2f}")
print(f"  {v_mm_per_year:.2f}")

The convection velocities are on the order of centimeters per year, similar to tectonic plate velocities!

In [ ]:
# Temperature in Kelvin
# Scalars are simpler - .min()/.max() return plain floats
T_dimensional_min = uw.quantity(T.min() * T0_scale, "K")
T_dimensional_max = uw.quantity(T.max() * T0_scale, "K")

print(f"Temperature range:")
print(f"  {T_dimensional_min:.1f}")
print(f"  {T_dimensional_max:.1f}")

## Compute Derived Quantities

We can compute other physically meaningful quantities from the solution:

In [ ]:
# Create variable for strain rate magnitude
strain_rate = uw.discretisation.MeshVariable("eps", mesh, 1, degree=1, units="1/s")

# Compute strain rate: ε̇ = √(½ε̇ᵢⱼε̇ᵢⱼ) where ε̇ᵢⱼ = ½(∂vᵢ/∂xⱼ + ∂vⱼ/∂xᵢ)
strain_rate_tensor = 0.5 * (v.diff(x) + v.diff(y).T)
strain_rate_expr = sympy.sqrt(
    0.5
    * (
        strain_rate_tensor[0, 0] ** 2
        + strain_rate_tensor[1, 1] ** 2
        + 2 * strain_rate_tensor[0, 1] ** 2
    )
)

# Project to mesh variable
proj = uw.systems.Projection(mesh, strain_rate)
proj.uw_function = strain_rate_expr
proj.solve()

print(f"Strain rate (ND): [{strain_rate.min():.3e}, {strain_rate.max():.3e}]")

In [ ]:
# Convert to dimensional units
eps0_scale = strain_rate.scaling_coefficient
strain_rate_dimensional = uw.quantity(strain_rate.max() * eps0_scale, "1/s")

# Convert to common geological units
strain_rate_per_year = strain_rate_dimensional.to("1/year")

print(f"Maximum strain rate: {strain_rate_per_year:.3e}")

## Summary

We successfully solved a thermal convection problem with non-dimensional scaling:

**Setup:**
- Set realistic mantle reference quantities
- Created variables with physical units
- Enabled ND scaling

**Physics:**
- Rayleigh number Ra ≈ 10⁷ (moderate convection)
- Coupled Stokes flow with temperature-dependent buoyancy
- Steady-state advection-diffusion balance

**Results:**
- PETSc worked with order-one ND values (good conditioning)
- Dimensional interpretation via scaling coefficients
- Velocities ~cm/year (realistic for mantle convection)
- Strain rates ~10⁻¹⁵ /s (typical geological rates)

### Key Insights

1. **Rayleigh Number**: The dimensionless parameter controlling convection vigor
2. **Buoyancy Scaling**: In ND form, F̂ = -Ra·T̂
3. **Coupled Systems**: Iterative solution for temperature-velocity coupling
4. **Dimensional Interpretation**: Use `uw.quantity(value * scale, units)` for unit conversion
5. **Vector Variables**: `.max()` returns tuple - use `max(v.max())` for largest component

### What's Next?

To extend this example:
- Add time evolution (time-dependent convection)
- Include viscosity variations (temperature-dependent rheology)
- Add internal heating (radioactive decay)
- Explore different boundary conditions
- Compute Nusselt number (heat flux efficiency)